In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import requests
import pymongo
from datetime import datetime
import ast

In [2]:
file = "data/Atlanta/Atlanta_CRIME_Data.csv"
data1 = pd.read_csv(file)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data1

,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,UCR #,IBR Code,Neighborhood,NPU,Latitude,Longitude
0,90010930,1/1/2009,1/1/2009,1145,1/1/2009,1148.0,408.0,NaN,NaN,2841 GREENBRIAR PKWY,Day Watch,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68845,-84.49328
1,90011083,1/1/2009,1/1/2009,1330,1/1/2009,1330.0,506.0,NaN,NaN,12 BROAD ST SW,Day Watch,9,LARCENY-NON VEHICLE,630,2303,Downtown,M,33.75320,-84.39201
2,90011208,1/1/2009,1/1/2009,1500,1/1/2009,1520.0,413.0,NaN,NaN,3500 MARTIN L KING JR DR SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Adamsville,H,33.75735,-84.50282
3,90011218,1/1/2009,1/1/2009,1450,1/1/2009,1510.0,204.0,NaN,NaN,3393 PEACHTREE RD NE,Evening Watch,8,LARCENY-NON VEHICLE,630,2303,Lenox,B,33.84676,-84.36212
4,90011289,1/1/2009,1/1/2009,1600,1/1/2009,1700.0,408.0,NaN,NaN,2841 GREENBRIAR PKWY SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68677,-84.49773
5,90011327,1/1/2009,1/1/2009,1645,1/1/2009,1645.0,607.0,NaN,NaN,1217 CAROLINE ST NE,Evening Watch,24,LARCENY-NON VEHICLE,630,2303,Edgewood,O,33.75786,-84.34875
6,90011450,1/1/2009,1/1/2009,1740,1/1/2009,1815.0,308.0,NaN,NaN,2685 METROPOLITAN PARKWAY,Evening Watch,12,LARCENY-NON VEHICLE,630,2303,Venetian Hills,S,33.70827,-84.45385
7,90011608,1/1/2009,1/1/2009,1627,1/1/2009,2110.0,204.0,NaN,NaN,3393 PEACHTREE RD,Evening Watch,8,LARCENY-NON VEHICLE,630,2303,Lenox,B,33.84676,-84.36212
8,90010721,1/1/2009,1/1/2009,815,1/1/2009,820.0,408.0,NaN,NaN,3030 HEADLAND DR SW,Day Watch,12,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68841,-84.48742
9,90010108,1/1/2009,12/31/2008,2000,1/1/2009,100.0,506.0,NaN,NaN,360 MARIETTA ST NW,Morning Watch,18,LARCENY-FROM VEHICLE,640,2305,Downtown,M,33.76269,-84.39702


In [4]:
# Column Names from csv
# ----------------------------
# Report Number	Report Date	Occur Date	Occur Time	Possible Date	Possible Time	Beat	
# Apartment Office Prefix	Apartment Number	Location	Shift Occurence	Location Type	
# UCR Literal	UCR #	IBR Code	Neighborhood	NPU	Latitude	Longitude

data = data1[["Report Number", "UCR #", "UCR Literal", "Occur Date", "Occur Time",  "Latitude", "Longitude"]].copy()
data.rename(columns={'Latitude':'latitude', 'Longitude':'longitude'}, inplace=True)
data["city"] = "atlanta"
data = data[["city", "Report Number", "UCR #", "UCR Literal", "Occur Date", "Occur Time",  "latitude", "longitude"]].copy()
data

,city,Report Number,UCR #,UCR Literal,Occur Date,Occur Time,latitude,longitude
0,atlanta,90010930,630,LARCENY-NON VEHICLE,1/1/2009,1145,33.68845,-84.49328
1,atlanta,90011083,630,LARCENY-NON VEHICLE,1/1/2009,1330,33.75320,-84.39201
2,atlanta,90011208,630,LARCENY-NON VEHICLE,1/1/2009,1500,33.75735,-84.50282
3,atlanta,90011218,630,LARCENY-NON VEHICLE,1/1/2009,1450,33.84676,-84.36212
4,atlanta,90011289,630,LARCENY-NON VEHICLE,1/1/2009,1600,33.68677,-84.49773
5,atlanta,90011327,630,LARCENY-NON VEHICLE,1/1/2009,1645,33.75786,-84.34875
6,atlanta,90011450,630,LARCENY-NON VEHICLE,1/1/2009,1740,33.70827,-84.45385
7,atlanta,90011608,630,LARCENY-NON VEHICLE,1/1/2009,1627,33.84676,-84.36212
8,atlanta,90010721,630,LARCENY-NON VEHICLE,1/1/2009,815,33.68841,-84.48742
9,atlanta,90010108,640,LARCENY-FROM VEHICLE,12/31/2008,2000,33.76269,-84.39702


In [5]:
# NA NAN Checking
num_NA = data.isna().sum()
print(num_NA)
print("\nPercentage of NA in whole data: \n" + str(num_NA/data["Report Number"].count()))
      

city             0
Report Number    0
UCR #            0
UCR Literal      0
Occur Date       0
Occur Time       0
latitude         0
longitude        0
dtype: int64

Percentage of NA in whole data: 
city             0.0
Report Number    0.0
UCR #            0.0
UCR Literal      0.0
Occur Date       0.0
Occur Time       0.0
latitude         0.0
longitude        0.0
dtype: float64


In [6]:
dates = []
years = []

for date in data["Occur Date"]:
    date = datetime.strptime(date, "%m/%d/%Y")
    dates.append(date)
    years.append(date.year)
    
data["date"] = dates
data["years"] = years
data

,city,Report Number,UCR #,UCR Literal,Occur Date,Occur Time,latitude,longitude,date,years
0,atlanta,90010930,630,LARCENY-NON VEHICLE,1/1/2009,1145,33.68845,-84.49328,2009-01-01,2009
1,atlanta,90011083,630,LARCENY-NON VEHICLE,1/1/2009,1330,33.75320,-84.39201,2009-01-01,2009
2,atlanta,90011208,630,LARCENY-NON VEHICLE,1/1/2009,1500,33.75735,-84.50282,2009-01-01,2009
3,atlanta,90011218,630,LARCENY-NON VEHICLE,1/1/2009,1450,33.84676,-84.36212,2009-01-01,2009
4,atlanta,90011289,630,LARCENY-NON VEHICLE,1/1/2009,1600,33.68677,-84.49773,2009-01-01,2009
5,atlanta,90011327,630,LARCENY-NON VEHICLE,1/1/2009,1645,33.75786,-84.34875,2009-01-01,2009
6,atlanta,90011450,630,LARCENY-NON VEHICLE,1/1/2009,1740,33.70827,-84.45385,2009-01-01,2009
7,atlanta,90011608,630,LARCENY-NON VEHICLE,1/1/2009,1627,33.84676,-84.36212,2009-01-01,2009
8,atlanta,90010721,630,LARCENY-NON VEHICLE,1/1/2009,815,33.68841,-84.48742,2009-01-01,2009
9,atlanta,90010108,640,LARCENY-FROM VEHICLE,12/31/2008,2000,33.76269,-84.39702,2008-12-31,2008


In [7]:
# data_2014 = pd.DataFrame()
# data_2015 = pd.DataFrame()
# data_2016 = pd.DataFrame()
# data_2017 = pd.DataFrame()
# data_2018 = pd.DataFrame()

# data_2014 = data.loc[data["years"] == 2014]
# print(data_2014)
# data_2015 = data.loc[data["years"] == 2015]
# print(data_2015)
# data_2016 = data.loc[data["years"] == 2016]
# print(data_2016)
# data_2017 = data.loc[data["years"] == 2017]
# print(data_2017)
# data_2018 = data.loc[data["years"] == 2018]
# data_2018

data = data.loc[(data["years"] >= 2014) & (data["years"] <= 2018)]
data

,city,Report Number,UCR #,UCR Literal,Occur Date,Occur Time,latitude,longitude,date,years
175621,atlanta,140011254,720,AUTO THEFT,1/1/2014,100,33.69915,-84.41819,2014-01-01,2014
175622,atlanta,140012074,341,ROBBERY-PEDESTRIAN,1/1/2014,1940,33.75290,-84.43904,2014-01-01,2014
175623,atlanta,140010429,311,ROBBERY-PEDESTRIAN,1/1/2014,226,33.77575,-84.46804,2014-01-01,2014
175624,atlanta,140010494,410,AGG ASSAULT,1/1/2014,245,33.76244,-84.41272,2014-01-01,2014
175625,atlanta,140011847,410,AGG ASSAULT,1/1/2014,1750,33.78554,-84.45441,2014-01-01,2014
175626,atlanta,140012191,410,AGG ASSAULT,1/1/2014,2000,33.75508,-84.45825,2014-01-01,2014
175627,atlanta,140010421,430,AGG ASSAULT,1/1/2014,215,33.70296,-84.45512,2014-01-01,2014
175628,atlanta,140010647,430,AGG ASSAULT,1/1/2014,410,33.75922,-84.44260,2014-01-01,2014
175629,atlanta,140010377,440,AGG ASSAULT,1/1/2014,200,33.84779,-84.36633,2014-01-01,2014
175633,atlanta,140010692,511,BURGLARY-RESIDENCE,1/1/2014,100,33.79333,-84.47416,2014-01-01,2014


In [8]:
def reformat_datetime(df):
    dates = []
    times = []

    for d,t  in zip(df["date"], df["Occur Time"]):
#         print(d)
#         print(type(d))
    #     datetime = (datetime.strftime(d, format='%Y-%m-%d') + " " + str(t))
        dy = datetime.strptime(str(d), "%Y-%m-%d %H:%M:%S")
        dy = datetime.strftime(dy, format='%Y-%m-%d') 
        ti = int(t/100)
    #     datetime = d + " " + t
        dates.append(dy)
        times.append(ti)
    df["date"] = dates
    df["time"] = times
    return df

# def reformat_datetime(df):
#     datetimes = []
#     for d in df["date"]:
#         date_time = datetime.strftime(d, format='%Y-%m-%dT%H:00:00')
#         datetimes.append(date_time)
#     df["date_time"] = datetimes
#     return df

In [9]:
data_atlanta = reformat_datetime(data)
data_atlanta

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,city,Report Number,UCR #,UCR Literal,Occur Date,Occur Time,latitude,longitude,date,years,time
175621,atlanta,140011254,720,AUTO THEFT,1/1/2014,100,33.69915,-84.41819,2014-01-01,2014,1
175622,atlanta,140012074,341,ROBBERY-PEDESTRIAN,1/1/2014,1940,33.75290,-84.43904,2014-01-01,2014,19
175623,atlanta,140010429,311,ROBBERY-PEDESTRIAN,1/1/2014,226,33.77575,-84.46804,2014-01-01,2014,2
175624,atlanta,140010494,410,AGG ASSAULT,1/1/2014,245,33.76244,-84.41272,2014-01-01,2014,2
175625,atlanta,140011847,410,AGG ASSAULT,1/1/2014,1750,33.78554,-84.45441,2014-01-01,2014,17
175626,atlanta,140012191,410,AGG ASSAULT,1/1/2014,2000,33.75508,-84.45825,2014-01-01,2014,20
175627,atlanta,140010421,430,AGG ASSAULT,1/1/2014,215,33.70296,-84.45512,2014-01-01,2014,2
175628,atlanta,140010647,430,AGG ASSAULT,1/1/2014,410,33.75922,-84.44260,2014-01-01,2014,4
175629,atlanta,140010377,440,AGG ASSAULT,1/1/2014,200,33.84779,-84.36633,2014-01-01,2014,2
175633,atlanta,140010692,511,BURGLARY-RESIDENCE,1/1/2014,100,33.79333,-84.47416,2014-01-01,2014,1





----

# Final view of date_time into weather data datetime format

In [10]:
data_atlanta.rename(columns={"UCR #": "code"}, inplace=True)
data_atlanta = data_atlanta[["city", "code", "date", "time", "latitude", "longitude"]].copy()
data_atlanta.to_csv (r'data/Atlanta/data_atlanta.csv', index = None, header=True)

data_atlanta

/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:4218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,city,code,date,time,latitude,longitude
175621,atlanta,720,2014-01-01,1,33.69915,-84.41819
175622,atlanta,341,2014-01-01,19,33.75290,-84.43904
175623,atlanta,311,2014-01-01,2,33.77575,-84.46804
175624,atlanta,410,2014-01-01,2,33.76244,-84.41272
175625,atlanta,410,2014-01-01,17,33.78554,-84.45441
175626,atlanta,410,2014-01-01,20,33.75508,-84.45825
175627,atlanta,430,2014-01-01,2,33.70296,-84.45512
175628,atlanta,430,2014-01-01,4,33.75922,-84.44260
175629,atlanta,440,2014-01-01,2,33.84779,-84.36633
175633,atlanta,511,2014-01-01,1,33.79333,-84.47416


In [11]:
# df_years = [data_2014, data_2015, data_2016, data_2017, data_2018]

# for i, yr in enumerate(df_years):
#     df = reformat_datetime(yr)
#     if i == 0:
#         data_2014_1 = df.copy()
#     elif i ==1:
#         data_2015_1 = df.copy()
#     elif i ==2:
#         data_2016_1 = df.copy()
#     elif i ==3:
#         data_2017_1 = df.copy()
#     elif i ==4:
#         data_2018_1 = df.copy()


In [ ]:
data_2014_1

In [ ]:
data_2015_1

In [ ]:
data_2016_1

In [ ]:
data_2017_1

In [ ]:
data_2018_1

In [ ]:
# .to_csv(index=False)
data_2014_1.to_csv (r'data/Atlanta/data_2014.csv', index = None, header=True)
data_2015_1.to_csv (r'data/Atlanta/data_2015.csv', index = None, header=True)
data_2016_1.to_csv (r'data/Atlanta/data_2016.csv', index = None, header=True)
data_2017_1.to_csv (r'data/Atlanta/data_2017.csv', index = None, header=True)
data_2018_1.to_csv (r'data/Atlanta/data_2018.csv', index = None, header=True)